# Training Spectral-Spatial Linear Transformer on Houston Dataset - Google Colab

This notebook trains the **Spectral-Spatial Linear Transformer** model on the Houston hyperspectral dataset.

## Model Architecture - NEW!
- **Spectral-Spatial Linear Attention**: Factorized attention with O(N) complexity
  - Linear spatial attention for efficient token mixing
  - Spectral gating mechanism for band correlations
- **Band-Weighted Pooling**: Learnable spectral band weighting (replaces simple pooling)
- **Global Attention Block**: Single full-attention block for long-range context
- **Ultra-low FLOPs and Latency**: Linear complexity for faster training and inference

## Dataset
- **Houston**: Hyperspectral image dataset
- **Spectral Bands**: 144
- **Classes**: 15 land-cover categories
- **Window Size**: 5×5 spatial patches

In [ ]:
# Install required packages
%pip install -q transformers scikit-learn seaborn einops thop scipy h5py

print("✓ Packages installed successfully!")


In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import scipy.io
import h5py
from torch.utils.data import Dataset
from transformers import TrainingArguments, Trainer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report, cohen_kappa_score, precision_score, recall_score, f1_score
import time
from einops import rearrange
from thop import profile
import os

print("✓ Libraries imported successfully!")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")


In [ ]:
# ============================================================
# NEW: Spectral-Spatial Linear Transformer Architecture
# ============================================================

# 1. Band-weighted pooling (NEW - replaces simple spectral attention)
class BandWeightedPooling(nn.Module):
    """
    Learnable spectral-band weighting for global token aggregation.
    Provides explicit spectral inductive bias.
    """
    def __init__(self, dim):
        super().__init__()
        self.weights = nn.Parameter(torch.ones(dim))

    def forward(self, x):
        # x: (B, N, C)
        w = torch.softmax(self.weights, dim=0)
        return (x * w).sum(dim=1)


# 2. Spectral-Spatial Factorized Linear Attention (CORE NOVELTY)
class SpectralSpatialLinearAttention(nn.Module):
    """
    Linear attention with explicit spectral gating.
    Spatial mixing via linear attention, spectral mixing via channel gate.
    O(N) complexity instead of O(N²)!
    """
    def __init__(self, dim, num_heads=8):
        super().__init__()
        assert dim % num_heads == 0
        self.num_heads = num_heads
        self.head_dim = dim // num_heads
        self.qkv = nn.Linear(dim, dim * 3, bias=False)
        self.proj = nn.Linear(dim, dim)
        
        # Spectral gate (explicit inductive bias)
        self.spectral_gate = nn.Sequential(
            nn.LayerNorm(dim),
            nn.Linear(dim, dim // 4),
            nn.GELU(),
            nn.Linear(dim // 4, dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        B, N, C = x.shape
        qkv = self.qkv(x).reshape(B, N, 3, self.num_heads, self.head_dim)
        q, k, v = qkv.unbind(dim=2)
        
        # Linear spatial attention
        k = k.softmax(dim=1)
        context = torch.einsum('bnhd,bnhv->bhdv', k, v)
        out = torch.einsum('bnhd,bhdv->bnhv', q, context)
        out = out.reshape(B, N, C)
        
        # Spectral gating
        gate = self.spectral_gate(x)
        out = out * gate
        
        return self.proj(out)


# 3. Global attention block (NEW - accuracy stabilizer)
class GlobalAttentionBlock(nn.Module):
    """
    Single full-attention block to restore long-range interactions.
    Cost is minimal, accuracy gain is large.
    """
    def __init__(self, dim, num_heads):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.attn = nn.MultiheadAttention(dim, num_heads, batch_first=True)

    def forward(self, x):
        h = self.norm(x)
        out, _ = self.attn(h, h, h)
        return x + out


# 4. Transformer Block (UPDATED)
class SpectralSpatialViTBlock(nn.Module):
    def __init__(self, dim, num_heads, mlp_ratio=4.):
        super().__init__()
        self.norm1 = nn.LayerNorm(dim)
        self.attn = SpectralSpatialLinearAttention(dim, num_heads)
        self.norm2 = nn.LayerNorm(dim)
        self.mlp = nn.Sequential(
            nn.Linear(dim, int(dim * mlp_ratio)),
            nn.GELU(),
            nn.Linear(int(dim * mlp_ratio), dim)
        )

    def forward(self, x):
        x = x + self.attn(self.norm1(x))
        x = x + self.mlp(self.norm2(x))
        return x


# 5. FINAL MODEL (NEW ARCHITECTURE)
class SpectralSpatialLinearTransformer(nn.Module):
    """
    Spectral-Spatial Linear Transformer for Hyperspectral Image Classification
    
    Key innovations:
    - Linear attention (O(N) complexity) with spectral gating
    - Band-weighted pooling for interpretable aggregation  
    - Single global attention for accuracy boost
    - Ultra-low FLOPs and latency
    """
    def __init__(
        self,
        image_size=5,
        patch_size=1,
        num_channels=103,
        num_classes=9,
        embed_dim=768,
        depth=6,
        num_heads=12,
        mlp_ratio=4.0
    ):
        super().__init__()
        if embed_dim % num_heads != 0:
            raise ValueError("embed_dim must be divisible by num_heads")
        
        # Patch embedding
        self.patch_embed = nn.Conv2d(
            num_channels,
            embed_dim,
            kernel_size=patch_size,
            stride=patch_size
        )
        num_patches_h = (image_size - patch_size) // patch_size + 1
        num_patches_w = (image_size - patch_size) // patch_size + 1
        num_patches = num_patches_h * num_patches_w
        self.pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
        
        # Linear attention blocks
        self.blocks = nn.ModuleList([
            SpectralSpatialViTBlock(embed_dim, num_heads, mlp_ratio)
            for _ in range(depth)
        ])
        
        # Single global attention block
        self.global_block = GlobalAttentionBlock(embed_dim, num_heads)
        
        # Spectral pooling + classifier
        self.spectral_pool = BandWeightedPooling(embed_dim)
        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, x, labels=None):
        # Patchify
        x = self.patch_embed(x)
        x = x.flatten(2).transpose(1, 2)
        x = x + self.pos_embed
        
        # Linear transformer blocks
        for blk in self.blocks:
            x = blk(x)
        
        # Global attention refinement
        x = self.global_block(x)
        
        # Spectral pooling
        x = self.spectral_pool(x)
        x = self.norm(x)
        logits = self.head(x)

        if labels is not None:
            loss = nn.CrossEntropyLoss()(logits, labels)
            return {"loss": loss, "logits": logits}
        return {"logits": logits}

# Alias for compatibility
newFastViT = SpectralSpatialLinearTransformer

print("✓ NEW Model architecture defined!")
print("  - Spectral-Spatial Linear Attention (O(N) complexity)")
print("  - Band-Weighted Pooling (interpretable)")
print("  - Global Attention Block (accuracy boost)")

## Step 2: Define Data Loader Functions


In [ ]:
# Data Loader Functions
def load_houston(image_file, gt_file):
    """
    Load Houston hyperspectral dataset from .mat files.
    Handles both MATLAB v7.3 (HDF5) and older formats.
    """
    print("Loading Houston dataset...")
    
    def load_mat_file(file_path):
        """Load .mat file, handling both v7.3 (HDF5) and older formats."""
        # First verify file exists and is readable
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"File not found: {file_path}")
        
        file_size = os.path.getsize(file_path)
        if file_size < 100:
            raise ValueError(f"File too small ({file_size} bytes), likely corrupted: {file_path}")
        
        # Check file header to determine format
        with open(file_path, 'rb') as f:
            header = f.read(4)
        
        # Try h5py first if it looks like HDF5 (MATLAB v7.3)
        if header == b'MATL' or header[:2] == b'\x00\x00':
            try:
                print(f"  Detected HDF5 format, using h5py...")
                f = h5py.File(file_path, 'r')
                keys = list(f.keys())
                print(f"  Found keys: {keys}")
                return f, keys, 'h5py'
            except Exception as e:
                print(f"  h5py failed: {e}, trying scipy...")
        
        # Try scipy for older MATLAB formats
        try:
            mat = scipy.io.loadmat(file_path)
            keys = [k for k in mat.keys() if not k.startswith('__')]
            print(f"  Found keys: {keys}")
            return mat, keys, 'scipy'
        except (ValueError, NotImplementedError) as e:
            # If scipy fails, try h5py as fallback
            print(f"  scipy.io.loadmat failed ({e}), trying h5py...")
            try:
                f = h5py.File(file_path, 'r')
                keys = list(f.keys())
                print(f"  Found keys: {keys}")
                return f, keys, 'h5py'
            except Exception as e2:
                # Provide helpful error message
                error_msg = f"Could not load {file_path}:\n"
                error_msg += f"  - scipy error: {e}\n"
                error_msg += f"  - h5py error: {e2}\n"
                error_msg += f"  - File size: {file_size} bytes\n"
                error_msg += f"  - File header: {header.hex()}\n"
                error_msg += "\nThe file might be corrupted or in an unsupported format.\n"
                error_msg += "Please verify the file was downloaded correctly."
                raise ValueError(error_msg)
    
    # Load both files
    image_mat, image_keys, image_format = load_mat_file(image_file)
    gt_mat, gt_keys, gt_format = load_mat_file(gt_file)
    
    print(f"  Image file format: {image_format}")
    print(f"  GT file format: {gt_format}")
    
    # Helper function to get data from either format
    def get_data(mat_obj, key, format_type):
        if format_type == 'scipy':
            return mat_obj[key]
        else:  # h5py
            # HDF5 files can store data directly or as references
            data_ref = mat_obj[key]
            
            # If it's a dataset, read it directly
            if isinstance(data_ref, h5py.Dataset):
                data = np.array(data_ref[:])
                return data
            
            # If it's a reference, follow it
            elif isinstance(data_ref, h5py.Reference):
                ref_obj = mat_obj[data_ref]
                if isinstance(ref_obj, h5py.Dataset):
                    return np.array(ref_obj[:])
                else:
                    return np.array(ref_obj)
            
            # If it's already an array-like object
            elif hasattr(data_ref, '__array__'):
                return np.array(data_ref)
            
            # Try to read as dataset
            else:
                try:
                    return np.array(data_ref[:])
                except:
                    return np.array(data_ref)
    
    # Load image data
    if len(image_keys) == 0:
        raise ValueError("No data keys found in image file.")
    elif len(image_keys) == 1:
        image_data = get_data(image_mat, image_keys[0], image_format)
        print(f"Using image data key: '{image_keys[0]}'")
    else:
        possible_image_keys = ['ori_data', 'houston', 'Houston', 'Houston13', 'data', 'image', 'HSI']
        image_data = None
        for key in possible_image_keys:
            if key in image_keys:
                image_data = get_data(image_mat, key, image_format)
                print(f"Found image data with key: '{key}'")
                break
        if image_data is None:
            image_data = get_data(image_mat, image_keys[0], image_format)
            print(f"Warning: Using first available key '{image_keys[0]}' from: {image_keys}")
    
    # Load ground truth data
    if len(gt_keys) == 0:
        raise ValueError("No data keys found in gt file.")
    elif len(gt_keys) == 1:
        ground_truth = get_data(gt_mat, gt_keys[0], gt_format)
        print(f"Using ground truth key: '{gt_keys[0]}'")
    else:
        possible_gt_keys = ['map', 'houston_gt', 'Houston_gt', 'Houston13_7gt', 'gt', 'ground_truth', 'label']
        ground_truth = None
        for key in possible_gt_keys:
            if key in gt_keys:
                ground_truth = get_data(gt_mat, key, gt_format)
                print(f"Found ground truth with key: '{key}'")
                break
        if ground_truth is None:
            ground_truth = get_data(gt_mat, gt_keys[0], gt_format)
            print(f"Warning: Using first available key '{gt_keys[0]}' from: {gt_keys}")
    
    # Ensure data is numpy array and handle transposition for HDF5
    if isinstance(image_data, np.ndarray):
        pass  # Already numpy array
    else:
        image_data = np.array(image_data)
    
    if isinstance(ground_truth, np.ndarray):
        pass  # Already numpy array
    else:
        ground_truth = np.array(ground_truth)
    
    # HDF5 files often store data transposed, check and fix if needed
    if image_format == 'h5py' and len(image_data.shape) == 3:
        # Check if dimensions look transposed (C, H, W instead of H, W, C)
        if image_data.shape[0] < image_data.shape[2]:
            image_data = np.transpose(image_data, (1, 2, 0))
            print("  Transposed image data from (C, H, W) to (H, W, C)")
    
    if gt_format == 'h5py' and len(ground_truth.shape) == 2:
        # Ground truth should be (H, W), check if transposed
        pass  # Usually correct for GT
    
    print(f"Image data shape: {image_data.shape}")
    print(f"Ground truth shape: {ground_truth.shape}")
    
    # Close HDF5 files if opened
    if image_format == 'h5py':
        image_mat.close()
    if gt_format == 'h5py':
        gt_mat.close()
    
    return image_data, ground_truth


def preprocess_data(image_data, ground_truth, window_size=5):
    """Preprocess hyperspectral data to extract spatial-spectral patches."""
    image_data = (image_data - np.min(image_data)) / (np.max(image_data) - np.min(image_data))
    padded_image = np.pad(image_data, ((window_size//2, window_size//2),
                                       (window_size//2, window_size//2),
                                       (0, 0)), mode='reflect')
    spatial_spectral_data = np.zeros((image_data.shape[0], image_data.shape[1],
                                      window_size, window_size, image_data.shape[2]))
    for i in range(image_data.shape[0]):
        for j in range(image_data.shape[1]):
            spatial_spectral_data[i, j] = padded_image[i:i+window_size, j:j+window_size, :]

    spatial_spectral_data = spatial_spectral_data.reshape(-1, window_size, window_size, image_data.shape[2])
    y = ground_truth.flatten()
    mask = y != 0
    spatial_spectral_data = spatial_spectral_data[mask]
    y = y[mask]

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)
    return spatial_spectral_data, y, label_encoder


class HyperspectralDataset(Dataset):
    def __init__(self, spatial_spectral_data, labels):
        self.spatial_spectral_data = spatial_spectral_data
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        feature = self.spatial_spectral_data[idx].transpose(2, 0, 1)
        label = self.labels[idx]
        return {
            'x': torch.tensor(feature, dtype=torch.float32),
            'labels': torch.tensor(label, dtype=torch.long)
        }

print("✓ Data loader functions defined!")


## Step 3: Define Utility Functions


In [ ]:
# Utility Functions
def calculate_latency_per_image(model, data_loader, device):
    model.eval()
    total_time, total_images = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch['x'].to(device)
            batch_size = inputs.shape[0]
            total_images += batch_size
            start_time = time.time()
            _ = model(inputs)
            total_time += (time.time() - start_time)
    return (total_time / total_images) * 1000

def calculate_throughput(model, data_loader, device):
    model.eval()
    total_samples, total_time = 0, 0
    with torch.no_grad():
        for batch in data_loader:
            inputs = batch['x'].to(device)
            batch_size = inputs.size(0)
            start_time = time.time()
            _ = model(inputs)
            total_time += time.time() - start_time
            total_samples += batch_size
    return total_samples / total_time

def overall_accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def average_accuracy(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    class_accuracies = cm.diagonal() / cm.sum(axis=1)
    return np.nanmean(class_accuracies)

def kappa_coefficient(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred)

def calculate_f1_precision_recall(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    return f1, precision, recall

def count_model_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad) / 1_000_000

def calculate_gflops(model, dataset, device):
    sample = dataset[0]['x'].unsqueeze(0).to(device)
    flops, _ = profile(model, inputs=(sample,), verbose=False)
    return flops / 1e9

print("✓ Utility functions defined!")


## Step 4: Download Houston Dataset


In [ ]:
# Create dataset directory
os.makedirs('dataset', exist_ok=True)

# Download Houston dataset with verification
print("Downloading Houston dataset...")
import urllib.request

def download_file(url, dest_path, description):
    """Download file with retry and verification."""
    max_retries = 3
    for attempt in range(max_retries):
        try:
            print(f"  Downloading {description}... (attempt {attempt + 1}/{max_retries})")
            urllib.request.urlretrieve(url, dest_path)
            
            # Verify file was downloaded and has content
            if os.path.exists(dest_path):
                size = os.path.getsize(dest_path)
                if size > 1000:  # At least 1KB
                    print(f"    ✓ Downloaded: {size / 1024:.1f} KB")
                    return True
                else:
                    print(f"    ✗ File too small ({size} bytes), retrying...")
                    os.remove(dest_path)
            else:
                print(f"    ✗ File not found, retrying...")
        except Exception as e:
            print(f"    ✗ Error: {e}, retrying...")
            if os.path.exists(dest_path):
                os.remove(dest_path)
    
    return False

# Try multiple download sources
download_sources = [
    {
        'image': "https://github.com/YuxiangZhang-BIT/Data-CSHSI/raw/main/datasets/Houston/Houston13.mat",
        'gt': "https://github.com/YuxiangZhang-BIT/Data-CSHSI/raw/main/datasets/Houston/Houston13_7gt.mat"
    },
    {
        'image': "https://raw.githubusercontent.com/YuxiangZhang-BIT/Data-CSHSI/main/datasets/Houston/Houston13.mat",
        'gt': "https://raw.githubusercontent.com/YuxiangZhang-BIT/Data-CSHSI/main/datasets/Houston/Houston13_7gt.mat"
    }
]

success = False
for i, source in enumerate(download_sources):
    print(f"\nTrying download source {i+1}...")
    if download_file(source['image'], "dataset/Houston13.mat", "Houston13.mat"):
        if download_file(source['gt'], "dataset/Houston13_7gt.mat", "Houston13_7gt.mat"):
            success = True
            break

if not success:
    print("\n⚠ Warning: Direct download failed. The files might be too large for GitHub raw links.")
    print("Please manually upload the Houston dataset files:")
    print("  1. Go to: https://github.com/YuxiangZhang-BIT/Data-CSHSI")
    print("  2. Navigate to datasets/Houston/")
    print("  3. Download Houston13.mat and Houston13_7gt.mat")
    print("  4. Upload them to Colab using: Files → Upload")
    print("  5. Move them to the dataset/ folder")
else:
    print("\n✓ Dataset downloaded successfully!")
    print("\nFile verification:")
    for f in os.listdir('dataset'):
        if f.endswith('.mat'):
            size = os.path.getsize(f'dataset/{f}') / 1024  # KB
            print(f"  {f}: {size:.1f} KB")
            
            # Check file header to verify format
            with open(f'dataset/{f}', 'rb') as file:
                header = file.read(4)
                if header == b'MATL':
                    print(f"    Format: MATLAB v7.3 (HDF5)")
                elif header[:2] == b'MI':
                    print(f"    Format: MATLAB v6/v7")
                else:
                    print(f"    Format: Unknown (header: {header.hex()})")


## Alternative: Manual File Upload

If automatic download fails, you can manually upload the files:

1. **Download files manually**:
   - Visit: https://github.com/YuxiangZhang-BIT/Data-CSHSI/tree/main/datasets/Houston
   - Download `Houston13.mat` and `Houston13_7gt.mat`
   - Or use direct links if available

2. **Upload to Colab**:
   - Use the file browser on the left sidebar
   - Click "Upload" and select the .mat files
   - Or use the code below to upload via dialog


In [ ]:
# Alternative: Upload files manually (uncomment if download failed)
# from google.colab import files
# import shutil

# print("Please upload Houston13.mat:")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     shutil.move(filename, f"dataset/{filename}")
#     print(f"Moved {filename} to dataset/")

# print("\nPlease upload Houston13_7gt.mat:")
# uploaded = files.upload()
# for filename in uploaded.keys():
#     shutil.move(filename, f"dataset/{filename}")
#     print(f"Moved {filename} to dataset/")

# Verify files exist
if os.path.exists("dataset/Houston13.mat") and os.path.exists("dataset/Houston13_7gt.mat"):
    print("✓ Dataset files found!")
    for f in ['Houston13.mat', 'Houston13_7gt.mat']:
        size = os.path.getsize(f'dataset/{f}') / (1024*1024)  # MB
        print(f"  {f}: {size:.2f} MB")
else:
    print("⚠ Dataset files not found. Please download or upload them.")


## Step 5: Load and Preprocess Dataset

**Note**: If you see errors about file format, the files might be corrupted. Try:
1. Re-downloading the files manually
2. Using the manual upload option above
3. Checking that files are actual .mat files (not HTML error pages)


## Step 5: Load and Preprocess Dataset


In [ ]:
# Load Houston dataset
image_file = "./dataset/Houston13.mat"
gt_file = "./dataset/Houston13_7gt.mat"

# First, verify files exist and check their content
print("Checking files...")
for file_path in [image_file, gt_file]:
    if os.path.exists(file_path):
        size = os.path.getsize(file_path)
        print(f"  {file_path}: {size / (1024*1024):.2f} MB")
        
        # Check if file might be an HTML error page (common with failed downloads)
        with open(file_path, 'rb') as f:
            first_bytes = f.read(200)
            if b'<!DOCTYPE' in first_bytes or b'<html' in first_bytes.lower() or b'404' in first_bytes:
                print(f"    ⚠ WARNING: File appears to be HTML (download may have failed)")
                print(f"    Please manually download and upload the file")
                print(f"    Visit: https://github.com/YuxiangZhang-BIT/Data-CSHSI/tree/main/datasets/Houston")
    else:
        print(f"  {file_path}: NOT FOUND")
        print(f"    Please download from: https://github.com/YuxiangZhang-BIT/Data-CSHSI/tree/main/datasets/Houston")

print("\nLoading dataset...")
try:
    image_data, ground_truth = load_houston(image_file, gt_file)
    print(f"\n✓ Dataset loaded successfully!")
    print(f"Image data shape: {image_data.shape}")
    print(f"Ground truth shape: {ground_truth.shape}")
except Exception as e:
    print(f"\n✗ Error loading dataset: {e}")
    print("\nTroubleshooting:")
    print("1. Verify files are valid .mat files (not HTML error pages)")
    print("2. Try manually uploading files using the upload cell above")
    print("3. Check file sizes - they should be several MB, not KB")
    print("4. Ensure files are in the dataset/ folder")
    raise


In [ ]:
# Preprocess data with 5x5 window size
window_size = 5
spatial_spectral_data, y, label_encoder = preprocess_data(image_data, ground_truth, window_size=window_size)

num_classes = len(np.unique(y))
num_channels = spatial_spectral_data.shape[-1]

print(f"\n✓ Data preprocessed successfully!")
print(f"Spatial-spectral data shape: {spatial_spectral_data.shape}")
print(f"Labels shape: {y.shape}")
print(f"Number of classes: {num_classes}")
print(f"Number of spectral bands: {num_channels}")
print(f"Class distribution: {np.bincount(y)}")


## Step 6: Split Dataset


In [ ]:
# Split dataset: 80% train, 20% test
train_indices, test_indices = train_test_split(
    np.arange(len(y)),
    test_size=0.2,
    stratify=y,
    random_state=42
)

train_dataset = HyperspectralDataset(spatial_spectral_data[train_indices], y[train_indices])
test_dataset = HyperspectralDataset(spatial_spectral_data[test_indices], y[test_indices])

print(f"✓ Dataset split successfully!")
print(f"Training samples: {len(train_dataset)}")
print(f"Testing samples: {len(test_dataset)}")
print(f"Train class distribution: {np.bincount(y[train_indices])}")
print(f"Test class distribution: {np.bincount(y[test_indices])}")


## Step 7: Initialize Model


In [ ]:
# Model configuration for Houston dataset
patch_size = 4
embed_dim = 192  # Must be divisible by num_heads
num_heads = 4
depth = 4

# Validate configuration
if embed_dim % num_heads != 0:
    raise ValueError(f"embed_dim ({embed_dim}) must be divisible by num_heads ({num_heads})")

# Calculate actual number of patches
actual_patches_h = (window_size - patch_size) // patch_size + 1
actual_patches_w = (window_size - patch_size) // patch_size + 1
actual_num_patches = actual_patches_h * actual_patches_w

print(f"Model Configuration:")
print(f"  Image size: {window_size}x{window_size}")
print(f"  Patch size: {patch_size}x{patch_size}")
print(f"  Actual patches: {actual_patches_h}x{actual_patches_w} = {actual_num_patches}")
print(f"  Number of spectral bands: {num_channels}")
print(f"  Number of classes: {num_classes}")
print(f"  Embed dim: {embed_dim} (divisible by num_heads={num_heads} ✓)")
print(f"  Head dim: {embed_dim // num_heads}")
print(f"  Depth: {depth}")

model = newFastViT(
    image_size=window_size,
    patch_size=patch_size,
    num_channels=num_channels,  # Houston: 144 bands
    num_classes=num_classes,     # Houston: 15 classes
    embed_dim=embed_dim,
    depth=depth,
    num_heads=num_heads,
    mlp_ratio=4.0
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print(f"\n✓ Model initialized successfully!")
print(f"Device: {device}")
print(f"Number of parameters: {count_model_parameters(model):.2f} M")

# Calculate GFLOPs
try:
    gflops = calculate_gflops(model, train_dataset, device)
    print(f"GFLOPs: {gflops:.2f}")
except Exception as e:
    print(f"Warning: Could not calculate GFLOPs: {e}")
    gflops = None


## Step 8: Setup Training


In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./results_houston_spectral_spatial",
    num_train_epochs=20,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_steps=100,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to="none",
    save_total_limit=3,
    metric_for_best_model="eval_loss",
    greater_is_better=False
)

# Data collator
def data_collator(data):
    return {
        'x': torch.stack([d['x'] for d in data]),
        'labels': torch.stack([d['labels'] for d in data])
    }

# Compute metrics function
def compute_metrics(p):
    predictions = p.predictions.argmax(-1)
    labels = p.label_ids
    accuracy = (predictions == labels).mean()
    return {"accuracy": accuracy}

# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

print("✓ Trainer setup complete!")


## Step 9: Train Model


In [ ]:
# Train the model
print("Starting training...")
print("This may take a while depending on your hardware...")
trainer.train()
print("✓ Training completed!")


## Step 10: Evaluate Model


In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print("\nEvaluation Results:")
for key, value in eval_results.items():
    print(f"{key}: {value:.4f}")


## Step 11: Generate Predictions and Calculate Metrics


In [ ]:
# Generate predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = y[test_indices]

# Calculate metrics
oa = overall_accuracy(y_true, y_pred)
aa = average_accuracy(y_true, y_pred)
kappa = kappa_coefficient(y_true, y_pred)
f1, precision, recall = calculate_f1_precision_recall(y_true, y_pred)

print("\n" + "="*50)
print("Classification Metrics")
print("="*50)
print(f"Overall Accuracy (OA):     {oa:.4f}")
print(f"Average Accuracy (AA):      {aa:.4f}")
print(f"Kappa Coefficient:          {kappa:.4f}")
print(f"F1 Score (weighted):         {f1:.4f}")
print(f"Precision (weighted):        {precision:.4f}")
print(f"Recall (weighted):           {recall:.4f}")
print("="*50)


## Step 12: Performance Metrics


In [ ]:
# Create test data loader for performance metrics
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

# Calculate performance metrics
latency = calculate_latency_per_image(model, test_loader, device)
throughput = calculate_throughput(model, test_loader, device)
params = count_model_parameters(model)

print("\n" + "="*50)
print("Performance Metrics")
print("="*50)
print(f"Latency per image:          {latency:.4f} ms")
print(f"Throughput:                  {throughput:.2f} samples/sec")
print(f"Model Parameters:            {params:.2f} M")
if gflops is not None:
    print(f"GFLOPs:                      {gflops:.2f}")
print("="*50)


## Step 13: Confusion Matrix


In [ ]:
# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(14, 12))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar_kws={'label': 'Count'})
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.title('Confusion Matrix - Houston Dataset', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

# Print classification report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, digits=4))


## Step 14: Per-Class Accuracy


In [ ]:
# Calculate per-class accuracy
class_accuracies = cm.diagonal() / cm.sum(axis=1)
class_names = [f"Class {i+1}" for i in range(num_classes)]

# Plot per-class accuracy
plt.figure(figsize=(12, 6))
bars = plt.bar(range(num_classes), class_accuracies, color='steelblue', alpha=0.7)
plt.xlabel('Class', fontsize=12)
plt.ylabel('Accuracy', fontsize=12)
plt.title('Per-Class Accuracy - Houston Dataset', fontsize=14, fontweight='bold')
plt.xticks(range(num_classes), class_names, rotation=45, ha='right')
plt.ylim([0, 1])
plt.grid(axis='y', alpha=0.3)

# Add value labels on bars
for i, (bar, acc) in enumerate(zip(bars, class_accuracies)):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{acc:.3f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

print("\nPer-Class Accuracies:")
for i, acc in enumerate(class_accuracies):
    print(f"  {class_names[i]}: {acc:.4f}")


## Summary

Training completed successfully! The model has been trained on the Houston hyperspectral dataset.

### Key Results:
- Overall Accuracy, Average Accuracy, and Kappa Coefficient are displayed above
- Confusion matrix shows per-class performance
- Performance metrics include latency, throughput, and model size

### Model Checkpoints:
The trained model is saved in `./results_houston/` directory. You can load it later using:
```python
from transformers import Trainer
trainer = Trainer.from_pretrained('./results_houston/checkpoint-<best>')
```

### Next Steps:
- Download the model checkpoints from Colab
- Use the trained model for inference on new data
- Experiment with different hyperparameters
